In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import time
import requests
from bs4 import BeautifulSoup
import pandas as pd

driver_path = 'C:/Users/Landon Hawley/Downloads/chromedriver-win64/chromedriver.exe'
service = Service(driver_path)

driver = webdriver.Chrome(service=service)

# Open the URL
url = "https://www.espn.com/nfl/stats/player/_/season/2020/seasontype/2"
driver.get(url)

time.sleep(5)

button = driver.find_element(By.LINK_TEXT, "Show More")
button.click()

# Allow time for the page to load
time.sleep(5)

button = driver.find_element(By.LINK_TEXT, "Show More")
button.click()

# Allow time for the page to load
time.sleep(5)

# Get the page source after JavaScript has loaded
html_content = driver.page_source

driver.quit()

In [2]:
soup = BeautifulSoup(html_content, "lxml")

namerank_columns = ['Rank', 'Name/Team']
namerank_df = pd.DataFrame(columns = namerank_columns)

namerank_div = soup.find(name = 'div', attrs = {'class' : 'ResponsiveTable ResponsiveTable--fixed-left mt4 Table2__title--remove-capitalization'})
table = namerank_div.find("table") 
tbody = table.find("tbody")  # Get the tbody element
rows = tbody.find_all("tr")  # Find all table rows

for row in rows:
    cols = row.find_all("td")  # Get all table data cells
    data = [col.text.strip() for col in cols]  # Extract text from each column
    namerank_series = pd.Series(data, index = namerank_columns)
    namerank_df = pd.concat([namerank_df, namerank_series.to_frame().T], ignore_index=True)

In [3]:
import re


team_abbrs = [
    "ARI", "ATL", "BAL", "BUF", "CAR", "CHI", "CIN", "CLE", "DAL", "DEN",
    "DET", "GB", "HOU", "IND", "JAX", "KC", "LAC", "LAR", "LV", "MIA",
    "MIN", "NE", "NO", "NYG", "NYJ", "PHI", "PIT", "SEA", "SF", "TB",
    "TEN", "WSH", "OAK"
]


team_pattern = r'(' + '|'.join(team_abbrs) + r')$'


def split_string_first(s):
    match = re.search(team_pattern, s)
    if match:
        return s[:match.start()]
    return s


def split_string_second(s):
    match = re.search(team_pattern, s)
    if match:
        return match.group()
    return ''


namerank_df['Name'] = namerank_df['Name/Team'].apply(split_string_first)
namerank_df['Team'] = namerank_df['Name/Team'].apply(split_string_second)
namerank_df.drop('Name/Team', inplace=True, axis=1)

In [4]:
stats_columns = ['Position', 'Games Played', 'Completions', 'Passing Attempts', 'Completion Percentage', 'Passing Yards',
                 'Avg Yards per Pass Attempt', 'Passing Yards per Game', 'Longest Pass', 'Passing Touchdowns', 'Interceptions',
                 'Total Sacks', 'Sack Yards Lost', 'Adjusted QBR', 'Passer Rating']
stats_df = pd.DataFrame(columns = stats_columns)

stats_div = soup.find(name = 'div', attrs = {'class' : 'Table__Scroller'})
table = stats_div.find("table") 
tbody = table.find("tbody")  # Get the tbody element
rows = tbody.find_all("tr")  # Find all table rows

for row in rows:
    cols = row.find_all("td")  # Get all table data cells
    data = [col.text.strip() for col in cols]  # Extract text from each column
    stats_series = pd.Series(data, index = stats_columns)
    stats_df = pd.concat([stats_df, stats_series.to_frame().T], ignore_index=True)

stats_df

,Position,Games Played,Completions,Passing Attempts,Completion Percentage,Passing Yards,Avg Yards per Pass Attempt,Passing Yards per Game,Longest Pass,Passing Touchdowns,Interceptions,Total Sacks,Sack Yards Lost,Adjusted QBR,Passer Rating
0,QB,16,382,544,70.2,"4,823",8.9,301.4,77,33,7,49,293,63.7,60.4
1,QB,15,390,588,66.3,"4,740",8.1,316.0,75,38,6,22,147,78.1,108.2
2,QB,16,401,610,65.7,"4,633",7.6,289.6,50,40,12,21,143,66.0,56.3
3,QB,16,407,626,65.0,"4,581",7.3,286.3,63,26,11,41,257,59.8,56.3
4,QB,16,396,572,69.2,"4,544",7.9,284.0,55,37,10,26,159,76.6,56.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112,WR,15,0,1,0.0,0,0.0,0.0,0,0,0,0,0,1.0,39.6
113,WR,12,0,0,0.0,0,0.0,0.0,0,0,0,1,16,0.0,39.6
114,WR,14,0,1,0.0,0,0.0,0.0,0,0,0,0,0,2.8,39.6
115,RB,14,0,0,0.0,0,0.0,0.0,0,0,0,1,4,0.0,39.6


In [5]:
total_stats = pd.concat([namerank_df, stats_df], axis=1)
total_stats

,Rank,Name,Team,Position,Games Played,Completions,Passing Attempts,Completion Percentage,Passing Yards,Avg Yards per Pass Attempt,Passing Yards per Game,Longest Pass,Passing Touchdowns,Interceptions,Total Sacks,Sack Yards Lost,Adjusted QBR,Passer Rating
0,1,Deshaun Watson,HOU,QB,16,382,544,70.2,"4,823",8.9,301.4,77,33,7,49,293,63.7,60.4
1,2,Patrick Mahomes,KC,QB,15,390,588,66.3,"4,740",8.1,316.0,75,38,6,22,147,78.1,108.2
2,3,Tom Brady,TB,QB,16,401,610,65.7,"4,633",7.6,289.6,50,40,12,21,143,66.0,56.3
3,4,Matt Ryan,ATL,QB,16,407,626,65.0,"4,581",7.3,286.3,63,26,11,41,257,59.8,56.3
4,5,Josh Allen,BUF,QB,16,396,572,69.2,"4,544",7.9,284.0,55,37,10,26,159,76.6,56.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112,102,DJ Moore,CAR,WR,15,0,1,0.0,0,0.0,0.0,0,0,0,0,0,1.0,39.6
113,102,Jeff Smith,NYJ,WR,12,0,0,0.0,0,0.0,0.0,0,0,0,1,16,0.0,39.6
114,102,Isaiah Wright,WSH,WR,14,0,1,0.0,0,0.0,0.0,0,0,0,0,0,2.8,39.6
115,102,James Robinson,JAX,RB,14,0,0,0.0,0,0.0,0.0,0,0,0,1,4,0.0,39.6


In [7]:
total_stats.drop(total_stats[total_stats['Name'].str.contains('/')].index, inplace = True)
total_stats

,Rank,Name,Team,Position,Games Played,Completions,Passing Attempts,Completion Percentage,Passing Yards,Avg Yards per Pass Attempt,Passing Yards per Game,Longest Pass,Passing Touchdowns,Interceptions,Total Sacks,Sack Yards Lost,Adjusted QBR,Passer Rating
0,1,Deshaun Watson,HOU,QB,16,382,544,70.2,"4,823",8.9,301.4,77,33,7,49,293,63.7,60.4
1,2,Patrick Mahomes,KC,QB,15,390,588,66.3,"4,740",8.1,316.0,75,38,6,22,147,78.1,108.2
2,3,Tom Brady,TB,QB,16,401,610,65.7,"4,633",7.6,289.6,50,40,12,21,143,66.0,56.3
3,4,Matt Ryan,ATL,QB,16,407,626,65.0,"4,581",7.3,286.3,63,26,11,41,257,59.8,56.3
4,5,Josh Allen,BUF,QB,16,396,572,69.2,"4,544",7.9,284.0,55,37,10,26,159,76.6,56.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112,102,DJ Moore,CAR,WR,15,0,1,0.0,0,0.0,0.0,0,0,0,0,0,1.0,39.6
113,102,Jeff Smith,NYJ,WR,12,0,0,0.0,0,0.0,0.0,0,0,0,1,16,0.0,39.6
114,102,Isaiah Wright,WSH,WR,14,0,1,0.0,0,0.0,0.0,0,0,0,0,0,2.8,39.6
115,102,James Robinson,JAX,RB,14,0,0,0.0,0,0.0,0.0,0,0,0,1,4,0.0,39.6


In [9]:
total_stats.to_csv("Passing_Stats_2020.csv", index = False)
print("Data saved to 2020_Passing_Stats.csv")

Data saved to 2020_Passing_Stats.csv
